```
From: https://github.com/ksatola
Version: 0.0.1

TODOs
1. 

```

# Local Development Environment with Docker and Visual Studio Code (VSC)

## Table of contents

- [Docker](#toc00)
- [Dockerize applications which rely on different services](#toc01)
- [Single container Python development environment for advanced analytics](#toc02)

---
<a id='toc00'></a>

# Docker

## Terminology
- **Images** - The blueprints of our application which form the basis of containers. In the demo above, we used the docker pull command to download the busybox image.
- **Containers** - Created from Docker images and run the actual application. We create a container using docker run which we did using the busybox image that we downloaded. A list of running containers can be seen using the docker ps command.
- **Docker Daemon** - The background service running on the host that manages building, running and distributing Docker containers. The daemon is the process that runs in the operating system which clients talk to.
- **Docker Client** - The command line tool that allows the user to interact with the daemon. More generally, there can be other forms of clients too - such as Kitematic which provide a GUI to the users.
- **Docker Hub** - A registry of Docker images. You can think of the registry as a directory of all available Docker images. If required, one can host their own Docker registries and can use them for pulling images.

### References
- https://docker-curriculum.com/
- https://www.digitalocean.com/community/tutorials/how-to-remove-docker-images-containers-and-volumes
- https://hub.docker.com/_/ubuntu
- https://docs.aws.amazon.com/elasticbeanstalk/latest/dg/eb-cli3.html
- https://docs.aws.amazon.com/elasticbeanstalk/latest/dg/docker.html
- https://stackoverflow.com/questions/23439126/how-to-mount-a-host-directory-in-a-docker-container
- https://stackoverflow.com/questions/27273412/cannot-install-packages-inside-docker-ubuntu-image
- https://www.python.org/doc/versions/

## Basics

```
#Fetch the busybox image from the Docker registry and saves it to our system
docker pull busybox

# List of images
docker images

# Run and exit a conainer
docker run busybox

# List of running containers
docker ps
docker ps -a

# Run a container in an interactive tty
docker run -it busybox sh
# exit

# Remove container
docker rm <container ID from docker ps -a> <another container ID from docker ps -a> ...

# Delete all containers that have a status of exited
# the -q flag, only returns the numeric IDs and -f filters output based on conditions provided
docker rm $(docker ps -a -q -f status=exited)
docker container prune

# Run and delete after exit
docker run --rm busybox

# Remove images
docker rmi busybox

# List untagged images
docker images -f dangling=true

# Remove all images
docker image prune
```

## Run a web server (with ports redirection)
```
docker run --rm prakhar1989/static-site

# Detached mode
docker run -d -P --name static-site prakhar1989/static-site
# -d will detach our terminal, -P will publish all exposed ports to random ports and finally --name corresponds to a name we want to give

docker port static-site

443/tcp -> 0.0.0.0:49153
443/tcp -> :::49153
80/tcp -> 0.0.0.0:49154
80/tcp -> :::49154

http://localhost:49154

# Specify a custom port to which the client will forward connections to the container
docker run -p 8888:80 prakhar1989/static-site

# Stop a detached container
docker stop static-site

# To deploy this on a real server you would just need to install Docker, and run the above Docker commands
```

## Docker images
```
# See the list of images that are available locally
docker images
# The TAG refers to a particular snapshot of the image and the IMAGE ID is the corresponding unique identifier for that image

# Minimal Ubuntu Docker images
# https://hub.docker.com/_/ubuntu
docker pull ubuntu:20.04
```

An important distinction to be aware of when it comes to images is the difference between base and child images:
- **Base images** are images that have no parent image, usually images with an OS like ubuntu, busybox or debian.
- **Child images** are images that build on base images and add additional functionality.

Then there are official and user images, which can be both base and child images:
- **Official images** are images that are officially maintained and supported by the folks at Docker. These are typically one word long. In the list of images above, the python, ubuntu, busybox and hello-world images are official images.
- **User images** are images created and shared by users like you and me. They build on base images and add additional functionality. Typically, these are formatted as user/image-name.

### Create and image
```
# Create an image that sandboxes a simple Flask application
cd ~/git
git clone https://github.com/prakhar1989/docker-curriculum.git
cd docker-curriculum/flask-app

# Create an image with this web app
# All user images are based on a base image. Since our application is written in Python, the base image we're going to use will be Python 3.
```
A **Dockerfile** is a simple text file that contains a list of commands that the Docker client calls while creating an image. It's a simple way to automate the image creation process. The best part is that the commands you write in a Dockerfile are almost identical to their equivalent Linux commands. This means you don't really have to learn new syntax to create your own dockerfiles.
```
# Create a new blank file in our favorite text-editor and save it in the same folder as the flask app by the name of Dockerfile

--- Dockerfile
# Base image
FROM python:3

# Set a directory for the app
WORKDIR /usr/src/app

# Copy all the files to the container
COPY . .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Tell the port number the container should expose (the app is working on this port)
EXPOSE 5000

# Run the command
# python ./app.py
CMD ["python", "./app.py"]

---

# Build our image
# The username should be the same one you created when you registered on Docker hub
# docker build takes an optional tag name with -t and a location of the directory containing the Dockerfile
docker build -t ksatola/catnip .

# Confirm the image is built
docker images

# Runk the image
docker run -p 8882:5000 ksatola/catnip
```

## Docker on AWS
```
# Use AWS Elastic Beanstalk to get our application up and running in a few clicks

One thing that I'd like to clarify before we go ahead is that it is not imperative to host your image on a public registry (or any registry) in order to deploy to AWS. In case you're writing code for the next million-dollar unicorn startup you can totally skip this step. The reason why we're pushing our images publicly is that it makes deployment super simple by skipping a few intermediate configuration steps.

# Publish our image on a registry which can be accessed by AWS
docker login
docker push ksatola/catnip

# See the published image
# https://hub.docker.com/u/ksatola

# Now that your image is online, anyone who has docker installed can play with your app by typing just a single command
docker run -p 8882:5000 ksatola/catnip
```
### Beanstalk
AWS Elastic Beanstalk (EB) is a PaaS (Platform as a Service) offered by AWS. If you've used Heroku, Google App Engine etc. you'll feel right at home. As a developer, you just tell EB how to run your app and it takes care of the rest - including scaling, monitoring and even updates. In April 2014, EB added support for running single-container Docker deployments which is what we'll use to deploy our app.
- Login to your [AWS console](https://signin.aws.amazon.com/signin?redirect_uri=https%3A%2F%2Fconsole.aws.amazon.com%2Fconsole%2Fhome%3Ffromtb%3Dtrue%26hashArgs%3D%2523%26isauthcode%3Dtrue%26state%3DhashArgsFromTB_us-east-1_b9c02ebe983036ce&client_id=arn%3Aaws%3Asignin%3A%3A%3Aconsole%2Fcanvas&forceMobileApp=0&code_challenge=hG1X_T64Dvy4gU7szjzQaCjDPLgbm2uDdkPPJ72r5yE&code_challenge_method=SHA-256).
- Click on Elastic Beanstalk. It will be in the compute section on the top left. Alternatively, you can access the [Elastic Beanstalk console](https://console.aws.amazon.com/elasticbeanstalk).
- Click on "Create New Application" in the top right
- Give your app a memorable (but unique) name and provide an (optional) description
- In the New Environment screen, create a new environment and choose the Web Server Environment.
- Fill in the environment information by choosing a domain. This URL is what you'll share with your friends so make sure it's easy to remember.
- Under base configuration section. Choose Docker from the predefined platform.
- Now we need to upload our application code. But since our application is packaged in a Docker container, we just need to tell EB about our container. Open the Dockerrun.aws.json file located in the flask-app folder and edit the Name of the image to your image's name. Don't worry, I'll explain the contents of the file shortly. When you are done, click on the radio button for "Upload your Code", choose this file, and click on "Upload".
```
{
  "AWSEBDockerrunVersion": "1",
  "Image": {
    "Name": "ksatola/catnip",
    "Update": "true"
  },
  "Ports": [
    {
      "ContainerPort": 5000,
      "HostPort": 8000
    }
  ],
  "Logging": "/var/log/nginx"
}
```
- Now click on "Create environment". The final screen that you see will have a few spinners indicating that your environment is being set up. It typically takes around 5 minutes for the first-time setup.
- Hopefully by now, our instance should be ready. Head over to the EB page and you should see a green tick indicating that your app is alive and kicking.
- Go ahead and open the URL in your browser and you should see the application in all its glory. Feel free to email / IM / snapchat this link to your friends and family so that they can enjoy a few cat gifs, too.

### Cleanup
Once you done basking in the glory of your app, remember to terminate the environment so that you don't end up getting charged for extra resources.

---
<a id='toc01'></a>

# Dockerize applications which rely on different services
Run and manage multi-container docker environments. Why multi-container? One of the key points of Docker is the way it provides isolation. The idea of bundling a process with its dependencies in a sandbox (called containers) is what makes this so powerful.

Just like it's a good strategy to decouple your application tiers, it is wise to keep containers for each of the services separate. Each tier is likely to have different resource needs and those needs might grow at different rates. By separating the tiers into different containers, we can compose each tier using the most appropriate instance type based on different resource needs. This also plays in very well with the whole microservices movement which is one of the main reasons why Docker (or any other container technology) is at the forefront of modern microservices architectures.

---
<a id='toc02'></a>

# Single container Python development environment for advanced analytics

## Dockerfile
```
# Base image
FROM ubuntu:20.04

# Set a directory for the app
#WORKDIR /home/ksatola

# Copy all the files to the container
COPY . .

# Install dependencies
RUN apt-get update && export DEBIAN_FRONTEND=noninteractive \
    && apt-get upgrade
    #&& apt-get -y install --no-install-recommends build-essential python3-pip git ruby-full
    
# Locales
RUN apt-get update && apt-get install -y locales && rm -rf /var/lib/apt/lists/* \
    && localedef -i en_US -c -f UTF-8 -A /usr/share/locale/locale.alias en_US.UTF-8
ENV LANG en_US.utf8

#RUN pip install --no-cache-dir -r requirements.txt

# Tell the port number the container should expose (the app is working on this port)
#EXPOSE 5000

# Run the command
# python ./app.py
#CMD ["python", "./app.py"]

```

## End-to-end procedure
```
# Run Docker Desktop in Windows
# Go to WSL Ubuntu terminal

cd ~/git
git clone https://github.com/ksatola/template-docker-ubuntu.git
cd template-docker-ubuntu

# Find a Dockerfile or create a new one
# There are pre-built Dockerfiles in the repo
touch Dockerfile<custom name>
vi Dockerfile
# i
# create content
# ESC
# :wq

docker build -t ksatola/ubuntu-dev-base .
docker images

# Run as many container as needed
# -t flag will prevent the container from exiting when running in the background
docker run -d -t -P --name ubuntu-tfx ksatola/ubuntu-dev-base

# Map a host directory to a docker container directory 
# https://stackoverflow.com/questions/23439126/how-to-mount-a-host-directory-in-a-docker-container
docker run -d -t -P --name ubuntu-tfx --mount src='/home/ksatola/git',target='/root/git',type=bind ksatola/ubuntu-dev-base

# To use host networking
docker run -d -t -P --network host --name ubuntu-tfx --mount src='/home/ksatola/git',target='/root/git',type=bind ksatola/ubuntu-dev-base

# ------------------------ BASE IMAGES
# !!!!!!!!! THIS ONE WORKS BEST !!!!!!!!!

# https://www.digitalocean.com/community/questions/how-to-install-a-specific-python-version-on-ubuntu
# https://www.python.org/doc/versions/

# Set A1
DOCKER_FILE='Dockerfile_python_base'
PYTHON_SYS_VERSION='3.7.0'
IMAGE_NAME='ksatola/ubuntu-dev-base-python3.7.0'

# Set A2
DOCKER_FILE='Dockerfile_python_base'
PYTHON_SYS_VERSION='3.5.0'
IMAGE_NAME='ksatola/ubuntu-dev-base-python3.5.0'

# Set A3
DOCKER_FILE='Dockerfile_python_base'
PYTHON_SYS_VERSION='3.8.12'
IMAGE_NAME='ksatola/ubuntu-dev-base-python3.8.12'

# Set A3
DOCKER_FILE='Dockerfile_python_base'
PYTHON_SYS_VERSION='3.9.7'
IMAGE_NAME='ksatola/ubuntu-dev-base-python3.9.7'

# Set A4
DOCKER_FILE='Dockerfile_python_from_image_base'
IMAGE_NAME='ksatola/ubuntu-python-dev-base'

docker build \
    -f $DOCKER_FILE \
    --build-arg PYTHON_SYS_VERSION=$PYTHON_SYS_VERSION \
    -t $IMAGE_NAME .

# ------------------------ EXTENDED IMAGES
# First create a base image and push it to Dockerhub (or use ones already uploaded)

# Set C1
DOCKER_FILE='Dockerfile_python_3.9.7_tfx1.2.0'
IMAGE_NAME='ksatola/ubuntu-dev-python3.9.7-tfx1.2.0'

docker build \
    -f $DOCKER_FILE \
    -t $IMAGE_NAME .

# ----------------------- CONTAINERS

# Set B1
IMAGE_NAME='ksatola/ubuntu-dev-base-python3.7.0'
CONTAINER_NAME='ubuntu-tfx-python3.7.0'

# Set B2
IMAGE_NAME='ksatola/ubuntu-dev-base-python3.8.12'
CONTAINER_NAME='ubuntu-tfx-python3.8.12'

# Set B3
IMAGE_NAME='ksatola/ubuntu-dev-python3.9.7-tfx1.2.0'
CONTAINER_NAME='ubuntu-python3.9.7-tfx1.2.0'

# Set B4
IMAGE_NAME='ksatola/ubuntu-python-dev-base'
CONTAINER_NAME='ubuntu-python-dev-base'


docker run -d -t -P \
    --name $CONTAINER_NAME \
    --mount src='/home/ksatola/git',target='/root/git',type=bind \
    $IMAGE_NAME



# ------------------------ DockerHub

docker push ksatola/ubuntu-dev-base-python3.7.0
docker push ksatola/ubuntu-dev-base-python3.8.12
docker push ksatola/ubuntu-dev-base-python3.9.7
docker push ksatola/ubuntu-dev-python3.9.7-tfx1.2.0


# ------------------------


# Connect to the container with VSC
# https://code.visualstudio.com/docs/remote/attach-container

F1-> Remote-Containers: Open Named Configuration File

{
  // Default path to open when attaching to a new container.
  //"workspaceFolder": "/path/to/code/in/container/here",
    "workspaceFolder": "/root",

  // An array of extension IDs that specify the extensions to
  // install inside the container when you first attach to it.
  "extensions": [
      "ms-python.python",
      "ms-python.vscode-pylance",
      "wholroyd.jinja",
  ],

  // Any *default* container specific VS Code settings
  //"settings": {
    //"terminal.integrated.shell.linux": "/bin/bash"
  //},

  // An array port numbers to forward
  //"forwardPorts": [8000],

  // Container user VS Code should use when connecting
  //"remoteUser": "vscode",
    "remoteUser": "root",

  // Set environment variables for VS Code and sub-processes
  //"remoteEnv": { "MY_VARIABLE": "some-value" }
}

# To run apt-get in the container, create a package cache
# https://stackoverflow.com/questions/27273412/cannot-install-packages-inside-docker-ubuntu-image
apt-get update
apt-get -y install curl
apt-get -qq -y install curl


# Jupyter Lab
# Run in VSC Terminal (the web interface will be avaiable via VSC and a web browser)
jupyter kernelspec list
jupyter --paths
jupyter lab --no-browser --allow-root
```